## Amazon SageMaker Feature Store: Introduction to Feature Store

This notebook demonstrates how to get started with Feature Store, create feature groups, and ingest data into them. These feature groups are stored in your Feature Store.

Feature groups are resources that contain metadata for all data stored in your Feature Store. A feature group is a logical grouping of features, defined in the feature store to describe records. A feature group’s definition is composed of a list of feature definitions, a record identifier name, and configurations for its online and offline store. 

### Overview
1. Set up
2. Creating a feature group
3. Ingest data into a feature group

### Prerequisites
This notebook uses sagemaker_core SDK and `Python 3 (Data Science)` kernel. This notebook works with Studio, Jupyter, and JupyterLab. 

#### Library dependencies:
* `sagemaker_core`
* `numpy`
* `pandas`

#### Role requirements:
**IMPORTANT**: You must attach the following policies to your execution role:
* `AmazonS3FullAccess`
* `AmazonSageMakerFeatureStoreAccess`

### Set up

In [ ]:
!pip uninstall sagemaker-core -y
!pip install pip --upgrade --quiet
!pip install sagemaker-core --upgrade

In [ ]:
import pandas as pd
import numpy as np
import io
import sagemaker
from sagemaker_core.helper.session_helper import get_execution_role, Session

sagemaker_session = Session()
REGION_NAME = sagemaker_session._region_name
role = get_execution_role()
s3_bucket_name = sagemaker.Session().default_bucket()
prefix = "sagemaker-featurestore-introduction"
default_bucket_prefix = sagemaker.Session().default_bucket_prefix

# If a default bucket prefix is specified, append it to the s3 path
if default_bucket_prefix:
    prefix = f"{default_bucket_prefix}/{prefix}"

### Inspect your data
In this notebook example we ingest synthetic data. We read from `./data/feature_store_introduction_customer.csv` and `./data/feature_store_introduction_orders.csv`.

In [ ]:
customer_data = pd.read_csv("data/feature_store_introduction_customer.csv")
orders_data = pd.read_csv("data/feature_store_introduction_orders.csv")

In [ ]:
customer_data.head()

In [ ]:
orders_data.head()

Below is an illustration on the steps the data goes through before it is ingested into a Feature Store. In this notebook, we illustrate the use-case where you have data from multiple sources and want to store them independently in a feature store. Our example considers data from a data warehouse (customer data), and data from a real-time streaming service (order data). 

![data flow](images/feature_store_data_ingest.svg)

### Create a feature group

We first start by creating feature group names for customer_data and orders_data. Following this, we create two Feature Groups, one for `customer_data` and another for `orders_data`

In [ ]:
from time import gmtime, strftime, sleep

customers_feature_group_name = "customers-feature-group-" + strftime("%d-%H-%M-%S", gmtime())
orders_feature_group_name = "orders-feature-group-" + strftime("%d-%H-%M-%S", gmtime())

In [ ]:
customers_feature_group_name

Instantiate a FeatureGroup object for customers_data and orders_data. 

In [ ]:
from sagemaker_core.shapes import FeatureDefinition

CustomerFeatureDefinitions = [
    FeatureDefinition(feature_name="customer_id", feature_type="Integral"),
    FeatureDefinition(feature_name="city_code", feature_type="Integral"),
    FeatureDefinition(feature_name="state_code", feature_type="Integral"),
    FeatureDefinition(feature_name="country_code", feature_type="Integral"),
    FeatureDefinition(feature_name="EventTime", feature_type="Fractional"),
]

OrderFeatureDefinitions = [
    FeatureDefinition(feature_name="customer_id", feature_type="Integral"),
    FeatureDefinition(feature_name="order_id", feature_type="Integral"),
    FeatureDefinition(feature_name="order_status", feature_type="Integral"),
    FeatureDefinition(feature_name="store_id", feature_type="Integral"),
    FeatureDefinition(feature_name="EventTime", feature_type="Fractional"),
]

In [ ]:
import time

current_time_sec = int(round(time.time()))

record_identifier_feature_name = "customer_id"

Append `EventTime` feature to your data frame. This parameter is required, and time stamps each data point.

In [ ]:
customer_data["EventTime"] = pd.Series([current_time_sec] * len(customer_data), dtype="float64")
orders_data["EventTime"] = pd.Series([current_time_sec] * len(orders_data), dtype="float64")

Load feature definitions to your feature group. 

Below we call create to create two feature groups, customers_feature_group and orders_feature_group respectively

In [ ]:
from sagemaker_core.shapes import OnlineStoreConfig, OfflineStoreConfig, S3StorageConfig
from sagemaker_core.resources import FeatureGroup

customers_feature_group = FeatureGroup.create(
    feature_group_name=customers_feature_group_name,
    record_identifier_feature_name=record_identifier_feature_name,
    event_time_feature_name="EventTime",
    role_arn=role,
    online_store_config=OnlineStoreConfig(enable_online_store=True),
    feature_definitions=CustomerFeatureDefinitions,
    offline_store_config=OfflineStoreConfig(
        s3_storage_config=S3StorageConfig(s3_uri=f"s3://{s3_bucket_name}/{prefix}")
    ),
)

In [ ]:
from sagemaker_core.resources import FeatureGroup

orders_feature_group = FeatureGroup.create(
    feature_group_name=orders_feature_group_name,
    record_identifier_feature_name=record_identifier_feature_name,
    event_time_feature_name="EventTime",
    role_arn=role,
    online_store_config=OnlineStoreConfig(enable_online_store=True),
    feature_definitions=OrderFeatureDefinitions,
    offline_store_config=OfflineStoreConfig(
        s3_storage_config=S3StorageConfig(s3_uri=f"s3://{s3_bucket_name}/{prefix}")
    ),
)

To confirm that your FeatureGroup has been created we use `wait_for_status` functions to wait for the feature group to be created successfully.

In [ ]:
def check_feature_group_status(feature_group):
    status = feature_group.wait_for_status(target_status="Created")
    print(f"FeatureGroup {feature_group.get_name()} successfully created.")


check_feature_group_status(customers_feature_group)
check_feature_group_status(orders_feature_group)

### Add metadata to a feature

We can add searchable metadata fields to FeatureGroup features by using the `FeatureMetadata` class. The currently supported metadata fields are `description` and `parameters`.

In [ ]:
from sagemaker_core.resources import FeatureMetadata
from sagemaker_core.shapes import FeatureParameter

customers_feature_metadata = FeatureMetadata(
    feature_group_name=customers_feature_group_name, feature_name="customer_id"
)

In [ ]:
customers_feature_metadata.update(
    description="The ID of a customer. It is also used in orders_feature_group.",
    parameter_additions=[FeatureParameter(key="idType", value="primaryKey")],
)

To view feature metadata, we can use `get` method to display that feature.

In [ ]:
customers_feature_metadata.get(
    feature_group_name=customers_feature_group_name, feature_name="customer_id"
)

Feature metadata fields are searchable. We use `search` API to find features with metadata that matches some search criteria.

In [ ]:
sagemaker_session.boto_session.client("sagemaker", region_name=region).search(
    Resource="FeatureMetadata",
    SearchExpression={
        "Filters": [
            {
                "Name": "FeatureGroupName",
                "Operator": "Contains",
                "Value": "customers-feature-group-",
            },
            {"Name": "Parameters.idType", "Operator": "Equals", "Value": "primaryKey"},
        ]
    },
)  # We use the boto client to search

### Ingest data into a feature group

We can put data into the FeatureGroup by using the `PutRecord` API. It will take < 1 minute to ingest data.

In [ ]:
# converting all columns to integral 64 type for further processing
customer_data["customer_id"] = pd.to_numeric(customer_data["customer_id"]).astype("Int64")
customer_data["city_code"] = pd.to_numeric(customer_data["city_code"]).astype("Int64")
customer_data["state_code"] = pd.to_numeric(customer_data["state_code"]).astype("Int64")
customer_data["country_code"] = pd.to_numeric(customer_data["country_code"]).astype("Int64")

orders_data["customer_id"] = pd.to_numeric(orders_data["customer_id"]).astype("Int64")
orders_data["order_id"] = pd.to_numeric(orders_data["order_id"]).astype("Int64")
orders_data["order_status"] = pd.to_numeric(orders_data["order_status"]).astype("Int64")
orders_data["store_id"] = pd.to_numeric(orders_data["store_id"]).astype("Int64")

Creating `IngestData` function to ingest all dataframe records using `PutRecord` API call.

In [ ]:
from sagemaker_core.shapes import FeatureValue


def IngestData(df, feature_group):
    try:
        feature_values_list = list()
        for index, row in df.iterrows():
            # Iterate through each column for the current row
            for column in df.columns:
                feature_values = FeatureValue(
                    feature_name=str(column), value_as_string=str(row[column])
                )
                feature_values_list.append(feature_values)
            feature_group.put_record(record=feature_values_list)
            feature_values_list.clear()
        SuccessString = f"The dataframe with {len(df)} rows has been ingested successfully for feature group {feature_group.get_name()}"
        return SuccessString
    except Exception as e:
        # Handle any other exceptions
        print(f"An unexpected error occurred: {e}")
        return None

In [ ]:
IngestData(customer_data, customers_feature_group)

In [ ]:
IngestData(orders_data, orders_feature_group)

Using an arbitrary customer record ID, 573291 we use `get_record` to check that the data has been ingested into the feature group.

In [ ]:
customer_id = 573291
sample_record = customers_feature_group.get_record(
    record_identifier_value_as_string=str(customer_id)
)

In [ ]:
sample_record

We use `batch_get_record` to check that all data has been ingested into two feature groups by providing customer IDs.

In [ ]:
from sagemaker_core.shapes import BatchGetRecordIdentifier

all_records_customers = customers_feature_group.batch_get_record(
    identifiers=[
        BatchGetRecordIdentifier(
            feature_group_name=customers_feature_group_name,
            record_identifiers_value_as_string=["573291", "109382", "828400", "124013"],
        )
    ]
)

In [ ]:
all_records_customers

In [ ]:
all_records_orders = orders_feature_group.batch_get_record(
    identifiers=[
        BatchGetRecordIdentifier(
            feature_group_name=orders_feature_group_name,
            record_identifiers_value_as_string=["573291", "109382", "828400", "124013"],
        )
    ]
)

In [ ]:
all_records_orders

### Add features to a feature group

If we want to update a FeatureGroup that has done the data ingestion, we can use the `Update` function and then re-ingest data by using the updated dataset.

In [ ]:
customers_feature_group.update(
    feature_additions=[
        FeatureDefinition(feature_name="email", feature_type="String"),
        FeatureDefinition(feature_name="name", feature_type="String"),
    ]
)
time.sleep(120)  # waiting for 120 seconds for the update process to get completed

Inspect the new dataset.

In [ ]:
customer_data_updated = pd.read_csv("data/feature_store_introduction_customer_updated.csv")

In [ ]:
customer_data_updated.head()

Append `EventTime` feature to your data frame again.

In [ ]:
customer_data_updated["EventTime"] = pd.Series(
    [current_time_sec] * len(customer_data), dtype="float64"
)

Ingest the new dataset.

In [ ]:
## need to see how to ingest data in new SDK
IngestData(customer_data_updated, customers_feature_group)

Use `batch_get_record` again to check that all updated data has been ingested into `customers_feature_group` by providing customer IDs.

In [ ]:
updated_records_customers = customers_feature_group.batch_get_record(
    identifiers=[
        BatchGetRecordIdentifier(
            feature_group_name=customers_feature_group_name,
            record_identifiers_value_as_string=["573291", "109382", "828400", "124013"],
        )
    ]
)

In [ ]:
updated_records_customers

### Clean up
Here we remove the Feature Groups we created. 

In [ ]:
customers_feature_group.delete()
orders_feature_group.delete()